# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
import os

from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
    ProfilerRule
)
from sagemaker.tuner import (
    CategoricalParameter, 
    ContinuousParameter, 
    HyperparameterTuner
)

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [4]:
#DONE: Fetch and upload the data to AWS S3

# Command to download and unzip data - comment out after having ran this code once
#!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
#!unzip dogImages.zip

# Upload data to AWS S3 

role = sagemaker.get_execution_role()
prefix = "project-image-classification"
bucket = "sagemaker-udacity-project3"
sagemaker_session = sagemaker.Session()
path = "dogImages"

#Comment out upload line after having ran once
#inputs = sagemaker_session.upload_data(path=path, bucket=bucket, key_prefix=prefix)



## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [5]:
#TODO: Declare your HP ranges, metrics etc. - done
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32,64,128,256,512]),
#    "epochs": CategoricalParameter([2,10,20])
}

objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex" : "Test Loss: ([0-9\\.]+)"}]

In [6]:
#TODO: Create estimators for your HPs

estimator = PyTorch(
    entry_point = "hpo.py",
    base_job_name = "Project4_dog_classifier_HPO",
    role = role,
    instance_count = 1,
    instance_type = "ml.m5.xlarge",
    py_version = 'py36',
    framework_version = '1.8'
) # TODO: Your estimator here - done

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs = 1,
    max_parallel_jobs = 1,
    objective_type = objective_type
) # TODO: Your HP tuner here - done

In [ ]:
# TODO: Fit your HP Tuner

training_path = "s3://{}/".format(bucket)
s3_output_dir = "s3://{}/{}/".format(bucket, "output")
s3_model_dir = "s3://{}/{}/".format(bucket, "model")

os.environ['SM_CHANNEL_TRAINING'] = training_path
os.environ['SM_MODEL_DIR'] = s3_model_dir
os.environ['SM_OUTPUT_DATA_DIR'] = s3_output_dir

#channel = {'training': "s3://sagemaker-udacity-project3/project-image-classification/"}
tuner.fit({'training': training_path}, wait = True) # TODO: Remember to include your data channels

.....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator() #TODO - done

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

In [ ]:
# hyperparameters = #TODO

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport())
]

hook_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

profiler_config = ProfilerConfig(system_monitor_interval_millis = 500, framework_profile_params = FrameworkProfile(num_steps = 1))


In [ ]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point = "train_model.py"
    base_job_name = "Project4 dog classifier"
    role = role,
    instance_count = 1,
    instance_type = "ml.m5.xlarge",
    py_version = 'py36',
    framework_version = '1.8'
    hyperparameters = hyperparameters,
    rules = rules,
    debugger_hook_config = hook_config,
    profiler_config = profile_config
)
estimator.fit(wait = False)
# estimator = estimator.attach(training_job_name)

# TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy(initial_instance_count=1, instance_type=gpu_instance_type) # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()